---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [9]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [5]:
def graph_identification():
    
    # Your Code Here
    sol = ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']
    
    return sol

graph_identification()


['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [10]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [53]:
def salary_predictions():
    
    raw_data = G.nodes(data = True)

    #Extract each feature from the network data
    Node_id = [a[0] for a in raw_data]
    Features = [b[1] for b in raw_data]
    Department = [c['Department'] for c in Features]
    Management_Salary = [d['ManagementSalary'] for d in Features]

    Degree = list(nx.degree_centrality(G).values())
    Closeness = list(nx.closeness_centrality(G).values())
    Betweenness = list(nx.betweenness_centrality(G, normalized = True).values())

    df1 = pd.DataFrame({'Node id': Node_id, 'Department': Department, 'Management Salary': Management_Salary, 
                        'Degree': Degree, 'Closeness': Closeness, 'Betweenness': Betweenness})
    data_df = df1[['Node id', 'Degree', 'Closeness', 'Betweenness', 'Department', 'Management Salary']]
    data_df = data_df.set_index('Node id')
    
    Final_test_df = data_df[data_df['Management Salary'].isnull() == True]
    Av_data_df = data_df[data_df['Management Salary'].isnull() == False]
    X_final_test = Final_test_df.drop('Management Salary', axis = 1)
    
    #Undersample available data so data becomes balanced
    One_data = Av_data_df[Av_data_df['Management Salary'] == 0].sample(119)
    Zero_data = Av_data_df[Av_data_df['Management Salary'] == 1]
    Data_under = pd.concat([One_data, Zero_data])
    Av_Data = Data_under.sample(frac = 1) 
    
    from sklearn.model_selection import train_test_split

    y_train = Av_Data['Management Salary']
    X_train = Av_Data[['Degree', 'Closeness', 'Betweenness', 'Department']]
    
    from sklearn.preprocessing import MinMaxScaler

    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_finaltest_scaled = scaler.transform(X_final_test)

    from sklearn.ensemble import RandomForestClassifier

    RF_Model = RandomForestClassifier()
    
    grid_values = {'max_depth':[2,3], 'min_samples_split':[2,3,4,5,6,7]}
        
    from sklearn.model_selection import GridSearchCV
    
    grid_RF = GridSearchCV(RF_Model, param_grid = grid_values, scoring = 'roc_auc', cv = 5)
    grid_RF.fit(X_train, y_train)
    
    res3 = np.array(grid_RF.cv_results_['mean_test_score'].reshape(6,2))
    res3

    clf = grid_RF.best_estimator_
    preds_prob = clf.predict_proba(X_final_test)[:,1]
    sol = pd.Series(preds_prob, index = Final_test_df.index)
    
    return sol

salary_predictions()


Node id
1       0.443061
2       0.990745
5       0.998039
8       0.538879
14      0.853947
18      0.541682
27      0.692662
30      0.540281
31      0.668612
34      0.205060
37      0.335706
40      0.574088
45      0.518876
54      0.668612
55      0.525445
60      0.544819
62      0.998039
65      0.990745
77      0.377428
79      0.517277
97      0.156784
101     0.280212
103     0.630017
108     0.580658
113     0.766972
122     0.181154
141     0.984862
142     0.998039
144     0.449326
145     0.634126
          ...   
913     0.156784
914     0.157474
915     0.069566
918     0.181154
923     0.069566
926     0.157474
931     0.156784
934     0.484534
939     0.069566
944     0.069566
945     0.069566
947     0.069566
950     0.571480
951     0.083852
953     0.156784
959     0.069566
962     0.069566
963     0.559253
968     0.069566
969     0.069566
974     0.069566
984     0.484534
987     0.083852
989     0.069566
991     0.134150
992     0.069566
994     0.069566
996   

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [60]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
print(future_connections.head(10))

            Future Connection
(6, 840)                  0.0
(4, 197)                  0.0
(620, 979)                0.0
(519, 872)                0.0
(382, 423)                0.0
(97, 226)                 1.0
(349, 905)                0.0
(429, 860)                0.0
(309, 989)                0.0
(468, 880)                0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [63]:
def new_connections_predictions():
    
    future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
    target_df = future_connections[future_connections['Future Connection'].isnull() == True]
    target_index = target_df.index
    
    # Your Code Here
    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']
    
    Pairs = future_connections.index
    future_connections['Node 1'] = [N[0] for N in Pairs]
    future_connections['Node 2'] = [N[1] for N in Pairs]
    future_connections['Common neighbors'] = [len(list(nx.common_neighbors(G, a, b))) for a,b in zip(future_connections['Node 1'], future_connections['Node 2'])]

    future_connections = future_connections.sort_values(['Node 1', 'Node 2'])
    
    Jaccard = list(nx.jaccard_coefficient(G))
    future_connections['Jaccard'] = [a[2] for a in Jaccard]
    
    R_allctn = list(nx.resource_allocation_index(G))
    P_attm = list(nx.preferential_attachment(G))

    Com_CN = list(nx.cn_soundarajan_hopcroft(G))
    Com_RA = list(nx.ra_index_soundarajan_hopcroft(G))

    future_connections['Resource'] = [a[2] for a in R_allctn]
    future_connections['PA'] = [a[2] for a in P_attm]
    future_connections['Com CN'] = [a[2] for a in Com_CN]
    future_connections['COM RA'] = [a[2] for a in Com_RA]
    
    Data_df = future_connections[['Common neighbors', 'Jaccard', 'Resource', 'PA', 'Com CN', 'COM RA', 'Future Connection']]

    Test_final = Data_df[Data_df['Future Connection'].isnull() == True]
    Av_data_df = Data_df[Data_df['Future Connection'].isnull() == False]

    X_final_test = Test_final.drop('Future Connection', axis = 1)
    
    #Undersample available data so data becomes balanced
    One_data = Av_data_df[Av_data_df['Future Connection'] == 0].sample(29332)
    Zero_data = Av_data_df[Av_data_df['Future Connection'] == 1]
    Data_under = pd.concat([One_data, Zero_data])
    Av_Data = Data_under.sample(frac = 1) 
    
    from sklearn.model_selection import train_test_split

    y_train = Av_Data['Future Connection']
    X_train = Av_Data[['Common neighbors', 'Jaccard', 'Resource', 'PA', 'Com CN', 'COM RA']]

    from sklearn.preprocessing import MinMaxScaler

    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_finaltest_scaled = scaler.transform(X_final_test)

    from sklearn.model_selection import GridSearchCV
    from sklearn.linear_model import LogisticRegression

    # Your code here
    Log_model = LogisticRegression()
        
    grid_values = {'penalty': ['l1', 'l2'], 'C':[0.01, 0.1, 1, 10, 100, 200, 500, 1000]}
    
    grid_log_model = GridSearchCV(Log_model, param_grid = grid_values, scoring = 'roc_auc', cv = 5)
    grid_log_model.fit(X_train_scaled, y_train)
    
    clf = grid_log_model.best_estimator_
    preds_prob = clf.predict_proba(X_finaltest_scaled)[:,1]
    sol = pd.Series(preds_prob, index = Test_final.index)
    
    return sol[target_index]

new_connections_predictions()


(107, 348)    0.248752
(542, 751)    0.118279
(20, 426)     0.931579
(50, 989)     0.118580
(942, 986)    0.118903
(324, 857)    0.118539
(13, 710)     0.658542
(19, 271)     0.652985
(319, 878)    0.118684
(659, 707)    0.118310
(49, 843)     0.118778
(208, 893)    0.118383
(377, 469)    0.105842
(405, 999)    0.178653
(129, 740)    0.173749
(292, 618)    0.181470
(239, 689)    0.118705
(359, 373)    0.111020
(53, 523)     0.720494
(276, 984)    0.118762
(202, 997)    0.118825
(604, 619)    0.354524
(270, 911)    0.118700
(261, 481)    0.464410
(200, 450)    0.999999
(213, 634)    0.118186
(644, 735)    0.317025
(346, 553)    0.117431
(521, 738)    0.115964
(422, 953)    0.178657
                ...   
(672, 848)    0.118700
(28, 127)     0.990370
(202, 661)    0.117302
(54, 195)     0.999935
(295, 864)    0.118492
(814, 936)    0.118232
(839, 874)    0.118903
(139, 843)    0.118518
(461, 544)    0.114928
(68, 487)     0.114746
(622, 932)    0.118419
(504, 936)    0.174283
(479, 528) 